## Information


In [ ]:
df.info()
display(df.describe()) 

# 특정조건만족하는 subset의 data info
display(df[df.Light <0].shape)

# df['date'].diff().dt.seconds.dropna() :  Data간의 차이를 초로 확인했을때
timedeltas = df['date'].diff().dt.seconds.dropna()  
display(timedeltas.describe())

# 데이터에 불균형이 있는지 확인
plt.figure(figsize=(15,4))
plt.scatter(df['date'],df['Occupancy'].astype('str'),s=0.03)
plt.show()

# Sorting하고, index는 지우겠다
tt = ttt.sort_values(['houseCode','date']).reset_index(drop=True) 

# column이름 바꾸기
tg = tg.rename(columns= {'power consumption':'power consumption sum'})

# data copy
tgg = tg.copy()

# 파생변수 만들기
tgg['c'] =tgg['houseCode'].str[-2:].astype('int')
tgg['d'] =tgg['date'].dt.hour
tgg['e'] =tgg['date'].dt.day

##### 'HouseCode'변수 Grouping / 'data'변수에서 15분 간격 / 'power consumption' 변수를 Sum으로 묶겠다
tt['date'] = pd.to_datetime(tt['date'])
tg = tt.groupby(['houseCode']).resample('15min', on='date')['power consumption'].sum().reset_index()
tg.head(n=5)

#### k-means로 군집변수 column 추가
from sklearn.cluster import KMeans 
# k-means clustering 실행
kmeans = KMeans(n_clusters=5)   
kmeans.fit(tgg.iloc[:,2:].values)   
tg['Cluster'] = kmeans.labels_

#### apply(suntimechecker) 를 원본 데이터 time 변수에 적용해서 -> suntime이라는 파생변수생성

def suntimeChecker(x):
    if pd.to_datetime(x).hour in list(range(6,18)):   # 객체 x의 시간이 오전6시 ~ 오후6시안에 포함된다면
        return 1 
    else:
        return 0

dt3['TIMESTAMP'] = pd.to_datetime(dt3['TIMESTAMP'])
dt3['suntime'] = dt3['TIMESTAMP'].apply(suntimeChecker) 


## Missing Value

###  1. 결측치 확인
- 비율, Count
- 시각화

In [ ]:

# Column 별 na 숫자 합
data.isnull().sum()

# 전체 개수 대비 많은 편일까? -> 각 열별 결측치의 비율
print(df.isna().sum()/len(df))

# embarked 열의 결측치 / 결측치 아닌것의 개수 
print(df['embarked'].isnull().sum())
print(df['embarked'].notnull().sum())

#### 2. 결측치 처리하기
##### 1  Drop na
- Drop 시키기 : dropna
- DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)

In [ ]:
# 결측이 있는 모든 행 삭제
data.dropna(how='any')

# 모든 열의 값이 결측일때 행을 삭제
data.dropna(how='all')

# thresh 매개변수는, 결측치가 아닌 데이터 개수의 기준
# th = 1이면, 결측치데이터가 1개 이상이면, 삭제
data.dropna(thresh=1)

# 결측치의 비율이 특정 이상인 행을 삭제
df.drop(columns = df.columns[df.isna().sum()/len(df) > 0.15], axis = 1)

#해당 칼럼을 기준으로 결측치를 처리한다.
data.dropna(subset=['칼럼이름'])

#####  2  Fill na
- df['column'].fillna(임의 값)
- df['column'].fillna(method='ffill' or 'bfill')
- DataFrame.fillna(self, value=None, method=None, axis=None, inplace=False, limit=None, downcast=None)

In [ ]:
# 0으로 대체하기
data.fillna(0)

# 평균값, 최소값, 분산, 표준편차, 평균등으로 특정 칼럼의 값을 대체하기
korea_data.fillna({'확진자':korea_data['확진자'].mean()})

# 행정구역의 결측치는 '서울'로, 국가/지역의 결측치는 '대한민국'으로 채우기
korea_data.fillna({'행정구역':'서울','국가/지역':'대한민국'})

In [ ]:
# 혼합해서 써보기
korea_data.fillna({'행정구역':'서울',
                   '국가/지역':'대한민국',
                   '확진자':korea_data['확진자'].mean()},
                  inplace=True)

In [ ]:
# 뒤의 값으로 채우기
df['age'].fillna(method = 'bfill', inplace = True)
df.iloc[190:200, 0:4]

# 앞의 값으로 채우기
df['deck'].fillna(method = 'ffill', inplace = True)
df.iloc[:10, 10:15]

##### 3  Interpolate 보간법

- df['age'].interpolate(inplace = True)

In [ ]:
## Piecewise Constant Interpolation
#  Linear Interpolation 이 주로 사용되고, 
#Piecewise Constant Interpolation 은 잘 사용되지 않는 편입니다. 
# 하지만 다차원의 다변량 보간 (in higher-dimensional multivariate interpolation)의 경우, 
# 속도와 단순성(speed and simplicity)  측면에서 선호하는 선택이 될 수 있습니다. 

f_nearest = interpolate.interp1d(
    x, y, kind='nearest')

y_new_nearest = f_nearest(xnew)

plt.plot(x, y, "o", xnew, y_new_nearest)
plt.show()

In [ ]:
#보간 값으로 넣기
df_interpolate.interpolate( method = 'linear', inplace = True)

def interpolated(df):
    temp=[]
    for i in df['ID'].unique():
        temp.append(df[df['ID']==i].interpolate())
    new_df=pd.concat(temp,axis=0)
    return new_df

import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

## generating the original data with missing values
x = np.arange(0, 4*np.pi, 0.5)
y = np.sin(x)

plt.plot(x, y, "o")
plt.show()

##1. 이전 값/ 다음 값을 이용하여 결측값 채우기 
# (Imputation using the previous/next values)
## Interpolation using the previous value
f_prev = interpolate.interp1d(
    x, y, kind='previous') # next
y_new_prev = f_prev(xnew)

plt.plot(x, y, "o", xnew, y_new_prev, '-')
plt.show()

In [ ]:
## 3 Linear Interpolation
f_linear = interpolate.interp1d(
    x, y, kind='linear')

y_new_linear = f_linear(xnew)

plt.plot(x, y, "o", xnew, y_new_linear, '-')
plt.show()

In [ ]:
## 4.  Spline Interpolation
f_quadr = interpolate.interp1d(
    x, y, kind='quadratic') # cubic

y_new_quadr = f_quadr(xnew)

plt.plot(x, y, "o", xnew, y_new_quadr)
plt.show()

In [ ]:
df_specific_value.fillna(0)  # 0으로 채워넣기

# 각 지역(우편번호)마다의 평균값을 준비 하기
mean_10054 = df[ 'avg_10054' ].mean()
# 준비한 평균값을으로 결측치를 대신하기
df_mean['avg_10054'].fillna(mean_10054, inplace = True)

# 직전 값으로 넣기
df_forward.fillna(method = 'pad', inplace = True)
# 직후 값으로 넣기
df_backward.fillna(method = 'bfill', inplace = True)


## Split, Concat, Merge

#### 1. Split

In [ ]:
df = pd.Dataframe(data)
v_split = df.variable.str.split('_', expand=True) 
v_split.head(3)

v_split.str.get(0) #왼쪽꺼 싹뽑기
v_statue = v_split.str.get(0)
v_country= v_split.str.get(1)
df['status'], df['country'] = (v_statue, v_country)

# 방법2
v_split = ebola_long.variable.str.split('_', expand=True) 
#위의 v_split이 데이터프레임이므로 컬럼명 바로 수정 가능
v_split.columns = ['status', 'country']
#곧바로 ebola_long과 v_split을 concat으로 연결
pd.concat([ebola_long, v_split], axis=1)

#### 2. Concat
- pd.concat([df1,df2,..],axis=0,ignore_index=False,join='outer')

In [ ]:
# 디폴트 -> row-bind로 붙여지는 게 기본형태
pd.concat([df1, df2, df3])

# 축을 바꿔보면? -> col-bind로 붙여짐 (같은 인덱스 번호인 애들끼리)
pd.concat([df1, df2, df3], axis = 1)

# 인덱스 번호 초기화 
pd.concat([df1, df2, df3], ignore_index = True)

# 참고로, col-bind로 붙여졌을 때도 인덱스 번호를 초기화할 수 있다
pd.concat([df1, df2, df3], axis = 1, ignore_index = True)

# 디폴트는 outer join으로 겹치는 값 상관없이 모두 join함.
# 겹치는 값만 뽑아보면 -> 열 이름이 같으므로 전부 뽑힌다!! 
pd.concat([df1, df2, df3], join = 'inner') #공통인 열만

In [ ]:
# 행 값이 1인 것만 추출 (인덱스가 1인 값 모두)
new_df.loc[1,]

# 이번에는 딱 1행만 추출
new_df.iloc[1, ]

In [ ]:
해당 행을, 기존 dataframe에 있는 column명들을 지정해서, concat으로 아래 붙여줄 수 있다

In [ ]:
## 컬럼이 여러개이면 Series가 아닌 Dataframe이므로 바꿔준다 
df4 = pd.DataFrame([[47, 92, 88, 76]], columns = ['Korean', 'Math', 'English', 'Science'])
pd.concat([new_df, df4])

#### 3. Merge
- (1) df1,df2 열 이름이 모두 동일한 경우 : pd.merge(df1,df2,on='공통열',how='inner')

- (2) df1,df2 열이 의미하는 것은 같은데 이름이 다른 경우 : pd.merge(df1,left_on='df1공통칼럼명',right_on='df2공통칼럼명',how='inner')


In [ ]:
pd.merge(df1,df2,on='공통열',how='inner')
pd.merge(df1,left_on='df1공통칼럼명',right_on='df2공통칼럼명',h

## 시각화
### Numerical
- 수치형 변수 1개 : 히스토그램, 분포도(hist+커널밀도함수), 박스플롯

##### 히스토그램(hist)

In [ ]:
plt.hist(tips['total_bill'])
## 다음의 코드도 동일한 그래프
#tips['total_bill'].hist()
#tips['total_bill'].plot.hist()

plt.show()

# bins 고정 : 구간의 수를 늘릴 수 있다
plt.hist(tips['total_bill'], bins = 20)
plt.show()

# 위는 y가 빈도수(freq)인데, 이를 y축 비율로 바꿈
plt.hist(tips['total_bill'], bins = 20, density = True)
plt.show()

##### 분포도(distplot)

In [ ]:
sns.distplot(tips['total_bill'])
plt.show()

# kde=False로 하면 히스토그램이 된다
sns.distplot(tips['total_bill'], kde = False, hist = True) #히스토그램 
plt.show()
# hist=False로 하면 커널밀도함수 그래프가된다
sns.distplot(tips['total_bill'], kde = True, hist = False) #커널밀도그래프
plt.show()

##### 박스플롯(Boxplot)

In [ ]:
sns.boxplot(tips['total_bill'])
plt.show()

# boxplot 세우기
sns.boxplot(y = 'total_bill', data = tips)
plt.show()

### 수치형 변수 2개 이상시각화(변수간 관계 확인)
#### 히트맵 sns.heatmap()

In [ ]:
tips.corr()
# 위의 테이블 형태를 그대로 sns.heatmap 안에 넣으면 끝
tip_cor = tips.corr()
sns.heatmap(tip_cor)

tip_pivot = pd.pivot_table(index = 'time', columns = 'size', values = 'tip', data = tips)
sns.heatmap(tip_pivot)


#### 라인플롯 sns.lineplot()

In [ ]:
# 두 수치형 데이터 size, tip 간 관계를 시각화
sns.lineplot(x = 'size', y = 'tip', data = tips)

# 참고로 hue에 범주형 데이터를 넣어서 선을 다양화할 수 있다.
# size와 tip의 라인플롯 - smoker 여부에 따라 달라지는지 체크
sns.lineplot(x= 'size', y = 'tip', hue = 'smoker', data = tips)

# 만일 x, y 둘 다 연속형을 쓰면,, 그림이 좀 이상해짐
sns.lineplot(x = 'total_bill', y = 'tip', data = tips)

#### 산점도 sns.scatterplot()
x,y둘다 연속형이면 산점도 쓰는걸 추천

In [ ]:
# 위에서 이상해보였던 total_bill과 tip 간 그래프를 산점도로 다시 그려보자 
sns.scatterplot(x = 'total_bill', y = 'tip', data = tips)

# 마찬가지로 hue에 범주형 변수를 넣어 시각화 가능
# total_bill과 tip 간 그래프를 성별에 따라 나눠 그리면
sns.scatterplot(x = 'total_bill', y = 'tip', data = tips, hue = 'sex')

# total_bill & tip 간 관계, 회귀 직선 추가
sns.regplot(x= 'total_bill', y = 'tip', data = tips)

# 옵션: 99% 신뢰구간으로 늘리면 
sns.regplot(x= 'total_bill', y = 'tip', data = tips, ci = 99)
# marker 모양 바꾸기
sns.regplot(x= 'total_bill', y = 'tip', data = tips, marker= '+')
sns.regplot(x = 'size', y = 'tip', data = tips, x_jitter = .1)

#### Pairplot


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.pairplot(df)
plt.show()

for v in df.select_dtypes(include='float'):
    target = df[v].dropna()
    plt.boxplot(target)
    plt.title(v)
    plt.show()
    
print('''
Humidity, HumidityRatio컬럼은 높은 상관관계를 보이며, 두 컬럼을 제외한 나머지 데이터의 경우 이상치가 상대적으로 많이 존재한다
''')

### Categorical
#### 카운터플롯(countplot) : 카테고리별 개수 확인

In [ ]:
sns.countplot(x = 'day', data = tips)

# bar의 위치 반대로
sns.countplot(y = 'day', data = tips)

#### 파이 그래프(plt.pie) : 범주형 데이터의 비율 구하기

In [ ]:
# 여성, 남성 비율을 구해보자.
pd.DataFrame(tips['sex'].value_counts()/tips['sex'].count())

# 비율과 레이블 리스트로 뽑아내기
new = pd.DataFrame(tips['sex'].value_counts()/tips['sex'].count())
new.reset_index(inplace = True)

# 위 리스트를 갖고 파이 그래프를 그리기
plt.pie(x = list(new['sex']), 
           labels = list(new['index']),
           autopct = '%.2f%%')
plt.show()

### 범주형 변수 2개 이상 시각화
#### 막대그래프(sns.barplot())
x축에는 범주형 데이터, y 축엔 범주값에 따른 데이터의 평균값이 구해지는 막대그래프

In [ ]:
# 성별 tip의 평균값
sns.barplot(x = 'sex', y = 'tip', data = tips)

# 성별 tip의 중위값
sns.barplot(x = 'sex', y = 'tip', data = tips, estimator = np.median)

## 그래프 가로로 그리기 -> x, y 위치 변경 
# 성별 tip의 평균값
sns.barplot(y = 'sex', x = 'tip', data = tips)

#### Pointplot
x축엔 범주형, y축엔 범주에 따른 수치형의 평균값
barplot은 bar모양으로, pointplot은 평균값이 점을 찍히고, category별 값이 선으로 연결된다.
(각 점 위아래에 길게 그려진 선은 신뢰구간이다.)

In [ ]:
# 요일(day)별 total_bill의 평균값
sns.pointplot(x = 'day', y = 'total_bill', data = tips)

# 요일별 total_bill의 최댓값 
sns.pointplot(x = 'day', y = 'total_bill', data = tips, estimator = np.max)

#### Boxplot
앞 두 그래프는 y축이 전부 평균값을 보여주지만, boxplot은 y축이 분포를 보여준다.
x 범주형, y축엔 범주에 따른 수치형 데이터의 분포 값

In [ ]:
# 식사 시간(time)별 total_bill의 분포
sns.boxplot(x = 'time', y = 'total_bill', data = tips)

# 그래프 눕히기 -> x, y 위치 변경 
sns.boxplot(y = 'time', x = 'total_bill', data = tips)

#### Violinplot : boxplot + kdeplot
boxplot은 분포를 확인하기는 좋지만, '분산'을 확인하기는 어렵다. 분산확인에 좋은게 바이올린 플롯이다.

In [ ]:
# time별 total_bill 분포
sns.violinplot(x= 'time', y = 'total_bill', data= tips)

# time별 total bill 분포 - 성별에 따라 다르게
sns.violinplot(x= 'time', y = 'total_bill', hue = 'sex', data = tips)

#split 버전
sns.violinplot(x= 'time', y = 'total_bill', hue = 'sex', data = tips, split = True)